### Transfer learning 
Training a neural network on traffic images to perform binary classification as:<br>
1) **medium** congested, classified as label **1**.<br>
2) **low** congested, classified as label **0**.<br>

Here we are using [Keras](https://github.com/keras-team/keras) with [TensorFlow](https://www.tensorflow.org/) backend to demostrate transfer learning on the [traffic images API](https://api.data.gov.sg/v1/transport/traffic-images) to classify images.
Training is first done on 9 images each from each class, then we 'freeze' the *feature layers* and rebuild the model to classify the remaining images.

After 87 images were scrapped each time from the [API](https://api.data.gov.sg/v1/transport/traffic-images) on 2 different times of the day - assuming one was during **peak time** and other was during **off-peak**, they had to be manually labelled.<br>

Images have been split to train and test (validation) set in the data folder. <br>

#### Import packages
Source: https://github.com/keras-team/keras/blob/master/examples/mnist_transfer_cnn.py

In [2]:
#from env py35
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from PIL import Image
import os
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

C:\Users\hafsa\AppData\Local\conda\conda\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


We will be using this later to record how long it takes to train our model.

In [2]:
now = datetime.datetime.now
print (now())

2018-12-21 02:02:40.948787


#### Instantiation of hyperparameters

Here we instantiate the values of hyperparameters that we will be using later. We will stick to the parameters similar to the deep neural network except for the `epochs` which is reduced, as CNN are more computationally expensive.

Some terms to define for my [reference](https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9):<br>
1) `batch_size`: total # of training examples present in the dataset<br>
2) `number of batches`: # of sets/parts a dataset is divided.<br>
3) `iterations`: # of batches needed to complete one epoch. AKA number of batches.<br>
4) `epoch`: when an entire dataset is passed once, forward and backward through the neural network.

We have 18 images in the training data. <br>
Batch size of 3 images are to be passed in 6 iterations/number of batches for one epoch.<br>
OR <br>
Batch size of 6 images are to be passed in 3 iterations/number of batches for one epoch.

In [1]:
# I will go with the first choice for the trial
batch_size = 3
num_classes = 6
epochs = 3

#### Labelling image data
As the scrapped data is not labelled, I decided which images to be considered as medium congestion and low congestion. The test data has a 50-50 balance of the medium vs low congestion.
Now, I would need to label the images as such that the image is labelled as class `0` or `1`.

In [3]:
DIR = './data/train'

# First, I need to figure how I should format the height x width image dimensions
# to input to a keras model
def get_size_statistics():
    heights = []
    widths = []
    img_count = 0
    for img in os.listdir(DIR):
        path = os.path.join(DIR, img)
        if "DS_Store" not in path:
            data = np.array(Image.open(path))
            heights.append(data.shape[0])
            widths.append(data.shape[1])
            img_count += 1
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))

get_size_statistics()

Average Height: 458.6666666666667
Max Height: 480
Min Height: 288


Average Width: 611.5555555555555
Max Width: 640
Min Width: 384
